# 01. 합성곱 신경망 (CNN)
> 이미지 분석 등에 주로 사용되는 합성곱 신경망(CNN)에 대해 알아봅시다.

- toc: true 
- badges: true
- comments: true
- categories: [Day 8]
- permalink: /cnn
- exec: colab

### 1. 이미지 프로세싱

이미지와 같은 데이터들은 2D 이상의 차원을 가집니다. 그러나 우리가 지금까지 배운 다층퍼셉트론(MLP)등의 모델은 이미지와 같은 2차원 데이터를 있는 그대로 처리하지 못합니다. 무슨말인지 알아볼까요? <br><br>

![](https://testerstories.com/files/ai_and_ml/ml-mnist-2d-to-1d.png)
<br>

우리가 기존에 사용하던 다층 퍼셉트론(MLP) 신경망은 데이터 샘플 하나를 벡터로서 입력받습니다. 우리가 붓꽃 예제를 풀때를 생각해봅시다. 하나의 데이터를 [꽃잎길이, 꽃잎너비, 꽃받침길이, 꽃받침너비]와 같이 Rank가 1인 벡터로 입력받아왔습니다. 이렇게 다층 퍼셉트론은 Rank가 1차원인 데이터를 입력받아야하기 때문에 이미지와 같은 높은 차원의 데이터를 1차원으로 평면화 시켜야합니다. 그러나 이렇게 평면화 시켜서 입력하면 이미지의 공간적인 특징이 사라지게 됩니다. 이 문제를 어떻게 해결했을까요? <br><br>

#### 1.1. 고양이의 눈에서 답을 얻다
![](https://i0.wp.com/k.kakaocdn.net/dn/nni6z/btqBD2aVqYg/uYXKGG1VB7LjZpua8OTOwK/img.png?w=750&ssl=1)

1950년대 허블(Hubel)과 비셀(Wiesel)은 고양이의 시각 피질 실험에서 고양이 시야의 한 쪽에 자극을 주었더니 전체 뉴런이 아닌 특정 뉴런만이 활성화되는 것을 발견했습니다. 즉, 눈이 어느 지점을 바라볼 때 뇌의 모든부분이 활성화 되는 것이 아니라 특정 부분만 활성화된다는 것이였습니다. 
<br><br>

이러한 아이디어에 영향을 받은 얀 르쿤(Yann Lecan) 교수는 기존처럼 모든 노드들이 전부 연결 되어있는 기존의 인공신경망이 아닌 눈이 바라보고 있는 곳에 따라서 특정 노드들만 연결되는 획기적인 인공신경망을 고안해 냈습니다. 이전 시간에 배운 RNN이 인간의 '기억' 메커니즘을 컴퓨터로 모델링했다면, CNN은 동물의 '눈'을 컴퓨터로 모델링한 신경망 구조입니다. <br><br>

#### 1.2. 이미지에 대해서

우리는 보통 이미지를 2차원 데이터라고 부르지만, 사실은 이는 정확한 말이 아닙니다. 흑백이미지의 경우는 2차원이 맞지만, 컬러 이미지는 3차원의 차원을 가집니다. 이게 무슨뜻일까요?

![](https://i0.wp.com/k.kakaocdn.net/dn/bbIGmT/btqBGftlKCn/yx8KQzY8N6tu8OZSyIrlTK/img.png?w=750&ssl=1)

컬러 이미지의 경우 가로, 세로말고 '채널'이라는 개념을 가지고 있습니다. 이 채널은 색상을 의미하는데 일반적으로 우리는 컬러 이미지가 R, G, B의 세개의 채널을 가지고 있다고 이야기합니다. 기존에 등장했던 신경망인 다층 퍼셉트론 신경망의 경우 3차원의 이미지 데이터를 학습시키기 위해 1차원으로 평면화 시키는데 이 과정에서 이미지의 공간 정보가 유실이 되게 됩니다. <br><br>

이미지의 공간 정보란 말 그대로 이미지가 가지는 ‘공간적인 정보’를 말합니다. 이미지를 이루는 가장 작은 단위를 픽셀(pixel)이라고 하는데 거리가 가까운 픽셀들은 서로 연관을 띠게 됩니다. 이는 곧 사진에서 음영, 선, 질감 등으로 보여지는 것이며 다시 말하면 이런 픽셀들의 연관성이 이미지가 가지는 공간정보라고 할 수 있습니다. CNN은 다층 퍼셉트론과 다르게 입력 데이터의 원래 구조를 유지해서 입력 받고 다음 단계로 전달할때에도 이 구조를 유지합니다. 이렇게 입력 데이터의 공간정보를 잃지 않으므로 이미지 데이터를 잘 이해할 수 있는 것입니다. <br><br>

#### 1.3. 필터에 대해서

이제 CNN의 첫번째 단계인 필터링 과정을 볼까요?. 필터링은 입력 받은 이미지를 잘 분류하기 위해 이미지에 필터를 씌워 특징을 찾는 작업입니다. 필터링은 이미지 처리 분야에서 광범위하게 이용되고 있는 기법으로써, 이미지에서 테두리 부분을 추출하거나 이미지를 흐릿하게 만드는 등의 기능을 수행하기 위해 이용됩니다. 필터링은 행렬의 형태로 표현된 이미지에 대해 행렬로 표현된 필터를 동일하게 적용함으로써 수행됩니다.  

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile9.uf.tistory.com%2Fimage%2F99EC29495C5D8ACF1CCF36)

이미지에 위와 같은 필터(행렬)을 **합성곱**하면 출력으로 오른쪽과 같은 이미지들이 출력됩니다. 사진편집 앱에서 이미지를 흐리게 하거나 날카롭게 하는 등의 기능을 보신적이 있으실겁니다. 그러한 기능들은 모두 위와 같은 필터를 이용해 수행하는 것입니다. 그렇다면 이렇게 만들어주는 합성곱 연산은 어떤 연산일까요?<br><br>

#### 1.4. 합성곱 (Convolution)

Convolution의 사전적 정의는 합성곱입니다. 단순히 이렇게만들으면 의미를 잘 이해하지 못할수 있는데 알고보면 이 알고리즘을 가장 잘 표현한 용어같습니다. 사실 Convolution은 처음 등장한 개념이 아니라 CNN이 등장하기 한참 전부터 이미지처리에서 사용되었던 개념입니다.

![](https://user-images.githubusercontent.com/15958325/58780750-defb7480-8614-11e9-943c-4d44a9d1efc4.gif)

위 gif의 첫번째 단계를 그림으로 보면 다음과 같습니다.

![](https://user-images.githubusercontent.com/15958325/58845860-ca23ed00-86b7-11e9-805f-ef5c8adcab9f.png)

빨간 박스는 필터가 적용될 영역이고 이름은 Patch(패치)라고 합니다. 또한 필터를 움직이는 크기는 개발자의 임의로 위 이미지처럼 한칸씩 이동시키면서 적용시킬건지, 두칸씩 이동할건지 정할 수 있습니다. 이 값을 stride라고 합니다.